This file reads the data that is generated from tensile tests. It first loads the relevant Python libaries, file information, and default variables that will be used in the program. It then stores all relevant data into 2D arrays. Finally, the program provides you a menu with which to access specific information.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import statsmodels.api as sm

#Information about the excel file:
file_name = "PT Ventures Spring 2024 - Tensile Data.xlsx"
sheets = ["Sachet Control", "5050 Sachet PP", "4060 Sachet PP", "3070 Sachet PP", "2080 Sachet PP", "1090 Sachet PP", "PP Control"]
header_row = 2

#Information about the tensile bar samples:
area = 0.0032 * 0.0123  #Area of tensile bars in meters squared
original_length = 0.057  #Gauge length of the tensile bars in meters

#Information about graph formatting:
downsample_factor = 10
marker_size = 2

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [2]:
#Important variables:
stresses = [[], [], [], [], [], [], []]
strains = [[], [], [], [], [], [], []]

yield_points = [[], [], [], [], [], [], []]  # Peak strain values
elasticites = [[], [], [], [], [], [], []]  # Slope of the linear line up to the yield point

In [3]:
def calculate_stress(force, area):
    return force / area

def calculate_strain(displacement, original_length):
    return displacement / original_length

def calculate_sheet_number(sheet_name):
    count = 0
    for sheet in sheets:
        if (sheet == sheet_name):
            return count
        count += 1

##############################################################################################################################################################################################  

def process_data(sheet_name):
    # Read the Excel file
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=header_row)
    trial_num = (df.shape[1] + 2) // 5  # Calculates the number of trials by dividing the number of rows by 5
    
    # Iterate over every 5th column and the one next to it, starting from column 2
    for i in range(1, 1 + trial_num * 5, 5):
        displacement_col = i
        force_col = i + 1

        # Extract displacement and force columns
        displacement_mm = df.iloc[:, displacement_col]  # displacement in mm
        force_kN = df.iloc[:, force_col]  # force in kN

        # Convert the units
        displacement = displacement_mm / 1000  # convert mm to meters
        force = force_kN * 1000  # convert kN to N

        # Calculate stress and strain
        stress = calculate_stress(force, area)
        strain = calculate_strain(displacement, original_length)

        #Storing relevant data
        sheet_num = calculate_sheet_number(sheet_name)
        stresses[sheet_num].append(stress)
        strains[sheet_num].append(strain)
        print(f"  {sheet_name}, trial {len(stresses[sheet_num])} stress/strain values stored")

def get_yield_points():
    #Iterating over the jagged, 2D array of stress vectors (stresses):
    for sheet in range(len(stresses)):  # 7 sheets for each sample type
        for trial in range(len(stresses[sheet])):  # 4 trials for now per sample type
            yield_point = max(stresses[sheet][trial])
            yield_points[sheet].append(yield_point)
            print(f"  {sheets[sheet]}, trial {trial+1} yield point values stored")

def get_elasticities():
    for sheet in range(len(stresses)):  # 7 sheets for each sample type
        for trial in range(len(stresses[sheet])):  # 4 trials for now per sample type
            stress_vector = stresses[sheet][trial]
            strain_vector = strains[sheet][trial]
            yield_point = yield_points[sheet][trial]
            
            yield_point_index = 0
            for i in range(len(stress_vector)):
                if (yield_point == stress_vector[i]):
                    break;
                yield_point_index += 1

            Y = stress_vector[0:yield_point_index]
            X = strain_vector[0:yield_point_index]
            
            model = sm.OLS(Y, X).fit()
            elasticity = model.params[0]  # The slope parameter from the model object
            elasticites[sheet].append(elasticity)
            print(f"  {sheets[sheet]}, trial {trial+1} elasticity values stored")

In [4]:
print("PROCESSING STRESS AND STRAIN VALUES")
for sheet in sheets:
    process_data(sheet)
    
print("\n\nPROCESSING YIELD POINT VALUES")
get_yield_points()

print("\n\nPROCESSING ELASTICITY VALUES")
get_elasticities()

PROCESSING STRESS AND STRAIN VALUES
  Sachet Control, trial 1 stress/strain values stored
  Sachet Control, trial 2 stress/strain values stored
  Sachet Control, trial 3 stress/strain values stored
  Sachet Control, trial 4 stress/strain values stored
  Sachet Control, trial 5 stress/strain values stored
  Sachet Control, trial 6 stress/strain values stored
  Sachet Control, trial 7 stress/strain values stored
  5050 Sachet PP, trial 1 stress/strain values stored
  5050 Sachet PP, trial 2 stress/strain values stored
  5050 Sachet PP, trial 3 stress/strain values stored
  5050 Sachet PP, trial 4 stress/strain values stored
  5050 Sachet PP, trial 5 stress/strain values stored
  5050 Sachet PP, trial 6 stress/strain values stored
  4060 Sachet PP, trial 1 stress/strain values stored
  4060 Sachet PP, trial 2 stress/strain values stored
  4060 Sachet PP, trial 3 stress/strain values stored
  4060 Sachet PP, trial 4 stress/strain values stored
  4060 Sachet PP, trial 5 stress/strain values

C:\Users\ringo\AppData\Local\Temp\ipykernel_7624\922423790.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elasticity = model.params[0]  # The slope parameter from the model object


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
def print_menu():
    print("\n1. Print average yield point values")
    print("2. Print average elasticity values")
    print("3. Display stress/strain curves")
    print("4. Quit\n")

##############################################################################################################################################################################################  

def print_yield_points():
    for sheet in range(len(yield_points)):  # 7 sheets for each sample type
        total = 0
        trial_num = len(yield_points[sheet])
        
        output = f"{sheets[sheet]} average yield point for {trial_num} trials: "
        for trial in range(len(yield_points[sheet])):  # 4 trials for now per sample type
            yield_point = yield_points[sheet][trial]
            total += yield_point
        
        average = total/trial_num
        output += f"{average:.2e}" + " Pa"
        print(output)

def print_elasticities():
    for sheet in range(len(elasticites)):  # 7 sheets for each sample type
        total = 0
        trial_num = len(elasticites[sheet])
        
        output = f"{sheets[sheet]} average elasticity for {trial_num} trials: "
        for trial in range(len(elasticites[sheet])):  # 4 trials for now per sample type
            elasticity = elasticites[sheet][trial]
            total += elasticity
        
        average = total/trial_num
        output += f"{average:.2e}" + " Pa"
        print(output)

def print_graphs():
    for sheet in sheets:
        print(f"{sheet} Stress/Strain Curve:")
        plt.figure(figsize=(8, 6))  # Creates a new figure for the plot
        
        sheet_num = calculate_sheet_number(sheet)
        for trial in range(len(stresses[sheet_num])):  # 7 sheets for each sample type
            stress_vector = stresses[sheet_num][trial]
            strain_vector = strains[sheet_num][trial]
            plt.plot(strain_vector[::downsample_factor] * 100, stress_vector[::downsample_factor], marker='o', markersize=marker_size, label=f"Trial {trial+1}")
        
        # Adds graph labels, title, and legend
        plt.title(f"Stress vs. Strain - {sheet}")
        plt.xlabel("Strain (%)")
        plt.ylabel("Stress (Pa)")
        plt.legend()

        # Format the axes
        plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x:.0f}%'))  #Uses an in-line function to format the x-axis as percentages (no decimals)
        plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:0.0e}'.format(y)))  #Uses an in-line function to format the y-axis in scientific notation
        plt.grid(True)

        #Showing the graph (with the combined plots of all trials)
        plt.show()

In [8]:
quitted = False
while not quitted:
    print_menu()
    choice = 0

    valid = False
    while not valid:
        choice = int(input("Please input a selection: "))
        if choice in [1, 2, 3, 4]:
            valid = True
        else:
            print("Input must be a \"1\", \"2\", \"3\", or \"4\"")
    
    if (choice == 1):  # Prints average yield points
        print_yield_points()
    elif (choice == 2):  # Prints average elasticities
        print_elasticities()
    elif (choice == 3):  #Displays stress/strain curves
        print_graphs()
    elif (choice == 4):  #Quits the program
        print("Quitting the program")
        quitted = True


1. Print average yield point values
2. Print average elasticity values
3. Display stress/strain curves
4. Quit

Quitting the program
